# Rodando C++ com CPLEX através do makefile criado

In [1]:
ls

AlphaTree.py                                 Notebook_Ideias.html
Arvore_AVL.py                                Notebook_Ideias.ipynb
CPUtimer.py                                  Notebook_tasks.html
Fibonacci_Heap.py                            Notebook_tasks.ipynb
instancia_500projetos_5sondas_delta_t84.dat  __pycache__/
instancias/                                  resultados.csv
instancias_testes_iniciais/                  solver_mestrado_CPLEX_CTP.cpp
Makefile*                                    solver_mestrado_CPLEX_TI*
makefile_template                            solver_mestrado_CPLEX_TI.cpp
modelo_TI_mestrado_joao.lp                   solver_mestrado_CPLEX_TI.o
Notebook_Codigo.ipynb                        synthetic_instance_generator.ipynb


In [2]:
! make clean

rm -rf *.o
rm -rf solver_mestrado_CPLEX_CTP
rm -rf solver_mestrado_CPLEX_TI


In [3]:
ls

AlphaTree.py                                 Notebook_Codigo.ipynb
Arvore_AVL.py                                Notebook_Ideias.html
CPUtimer.py                                  Notebook_Ideias.ipynb
Fibonacci_Heap.py                            Notebook_tasks.html
instancia_500projetos_5sondas_delta_t84.dat  Notebook_tasks.ipynb
instancias/                                  __pycache__/
instancias_testes_iniciais/                  resultados.csv
Makefile*                                    solver_mestrado_CPLEX_CTP.cpp
makefile_template                            solver_mestrado_CPLEX_TI.cpp
modelo_TI_mestrado_joao.lp                   synthetic_instance_generator.ipynb


In [4]:
ls instancias/

instancia_100projetos_10sondas_delta_t28.dat
instancia_100projetos_10sondas_delta_t7.dat
instancia_100projetos_10sondas_delta_t84.dat
instancia_100projetos_2sondas_delta_t1.dat
instancia_100projetos_2sondas_delta_t28.dat
instancia_100projetos_2sondas_delta_t7.dat
instancia_100projetos_2sondas_delta_t84.dat
instancia_100projetos_5sondas_delta_t28.dat
instancia_100projetos_5sondas_delta_t7.dat
instancia_100projetos_5sondas_delta_t84.dat
instancia_10projetos_2sondas_delta_t1.dat
instancia_10projetos_2sondas_delta_t28.dat
instancia_10projetos_2sondas_delta_t7.dat
instancia_10projetos_2sondas_delta_t84.dat
instancia_200projetos_10sondas_delta_t28.dat
instancia_200projetos_10sondas_delta_t7.dat
instancia_200projetos_10sondas_delta_t84.dat
instancia_200projetos_20sondas_delta_t28.dat
instancia_200projetos_20sondas_delta_t7.dat
instancia_200projetos_20sondas_delta_t84.dat
instancia_200projetos_2sondas_delta_t1.dat
instancia_200projetos_2sondas_delta_t28.dat
instancia_200projetos_2sondas_delta_

In [5]:
! make ti ARGS=instancias/instancia_10projetos_2sondas_delta_t84.dat

g++ -O0 -c -m64 -O -fPIC -fno-strict-aliasing -fexceptions -DNDEBUG -DIL_STD -I/opt/ibm/ILOG/CPLEX_Studio1210/cplex/include -I/opt/ibm/ILOG/CPLEX_Studio1210/concert/include solver_mestrado_CPLEX_TI.cpp -o solver_mestrado_CPLEX_TI.o 
In file included from /opt/ibm/ILOG/CPLEX_Studio1210/concert/include/ilconcert/iloanyset.h:22,
                 from /opt/ibm/ILOG/CPLEX_Studio1210/concert/include/ilconcert/ilomodel.h:26,
                 from /opt/ibm/ILOG/CPLEX_Studio1210/cplex/include/ilcplex/ilocplex.h:27,
                 from solver_mestrado_CPLEX_TI.cpp:5:
/opt/ibm/ILOG/CPLEX_Studio1210/concert/include/ilconcert/iloany.h:32:24: warning: ignoring attributes on template argument ‘IloAny’ {aka ‘void*’} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wignored-attributes-Wignored-attributes]8;;]
   32 | typedef IloArray<IloAny> IloAnyArrayBase;
      |                        ^
In file included from /opt/ibm/ILOG/CPLEX_Studio1210/concert/include/ilconcert/ilomodel.h

In file included from /opt/ibm/ILOG/CPLEX_Studio1210/concert/include/ilconcert/ilosolution.h:34,
                 from /opt/ibm/ILOG/CPLEX_Studio1210/concert/include/ilconcert/ilomodel.h:839,
                 from /opt/ibm/ILOG/CPLEX_Studio1210/cplex/include/ilcplex/ilocplex.h:27,
                 from solver_mestrado_CPLEX_TI.cpp:5:
/opt/ibm/ILOG/CPLEX_Studio1210/concert/include/ilconcert/ilohash.h:589:69: warning: ignoring attributes on template argument ‘IloAny’ {aka ‘void*’} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wignored-attributes-Wignored-attributes]8;;]
  589 | class IloAddressHashTable : public IloEnvHashTable<IloAny, ValueType> {
      |                                                                     ^
/opt/ibm/ILOG/CPLEX_Studio1210/concert/include/ilconcert/ilohash.h: In constructor ‘IloAddressHashTable<ValueType>::IloAddressHashTable(IloMemoryManager, IloInt)’:
/opt/ibm/ILOG/CPLEX_Studio1210/concert/include/ilconcert/ilohash.h:592:37: war

# Metaheurísticas

## Lendo dados

In [6]:
def Read_data(path):
    
    import copy
    
    coords_x = []
    coords_y = []
    bacias = []
    nomes = []
    maturidades = []
    qualidades = []
    plays = []
    soterramentos = []
    pcgnas = []
    geracao = []
    migracao = []
    reservatorio = []
    geometria = []
    retencao = []
    pshc = []
    mc_vol = []
    mi_vol = []
    mc_vpl = []
    mi_vpl = []
    custos = []
    tempos_exec = []
    inicio_janela = []
    final_janela = []
    sondas_x = []
    sondas_y = []
    
    data = {
        1: coords_x,
        2: coords_y,
        3: bacias,
        4: nomes,
        5: maturidades,
        6: qualidades,
        7: plays,
        8: soterramentos,
        9: pcgnas,
        10: geracao,
        11: migracao,
        12: reservatorio,
        13: geometria,
        14: retencao,
        15: pshc,
        16: mc_vol,
        17: mi_vol,
        18: mc_vpl,
        19: mi_vpl,
        20: custos,
        21: tempos_exec,
        22: inicio_janela,
        23: final_janela,
        25: sondas_x,
        26: sondas_y
    }
    
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if i == 0:
                n_projetos = int(line.split()[0])
                custo_total = float(line.split()[1])
                t_init = int(line.split()[2])
                t_final = int(line.split()[3])
                delta_t = int(line.split()[4])
                n_periodos = int(line.split()[5])
            elif i == 24:
                n_sondas = int(line.split()[0])
            elif i <= 26:
                for elem in line.split('[')[1].split(']')[0].split(','):
                    try:
                        data[i].append(float(elem))
                    except:
                        data[i].append(elem)
            elif i > 26:
                setup = []
                for k, l in enumerate(line.split('[[')[1].split('], [')):
                    setup.append([])
                    for elem in l.split(']]')[0].split(', '):
                        setup[k].append(float(elem))
    
    return n_projetos, n_sondas, n_periodos, t_init, t_final, delta_t, custo_total, data, setup


## Classe para lista duplamente encadeada

In [7]:
class Node:
    
    """
    Documentação:
    """
    
    def __init__(self, data=None):
        self.data = data
        self.next = None
        self.prev = None
    

class DoublyLinkedList:
    
    """
    Documentação:
    """
    
    def __init__(self, nodes=None):
        self.head = None
        if nodes is not None:
            node = Node(data=nodes.pop(0))
            self.head = node
            for elem in nodes:
                new_node = Node(data=elem)
                node.next = new_node
                new_node.prev = node
                node = node.next
    
    def __iter__(self):
        node = self.head
        while node is not None:
            yield node
            node = node.next
    
    def add_node_begining(self, value_to_add):
        new_node = Node(data=value_to_add)
        
        if not self.head:
            self.head = new_node
            return
        
        new_node.next = self.head
        self.head.prev = new_node
        self.head = new_node
    
    def add_node_end(self, value_to_add):
        
        new_node = Node(data=value_to_add)
        
        if not self.head:
            self.head = new_node
            return
        
        for curr in self:
            pass
        curr.next = new_node
        new_node.prev = curr
    
    def add_node_after_node(self, value_to_add, node_ref):
        new_node = Node(data=value_to_add)
        new_node.next = node_ref.next
        new_node.prev = node_ref
        node_ref.next.prev = new_node
        node_ref.next = new_node
    
    def add_node_before_node(self, value_to_add, node_ref):
        new_node = Node(data=value_to_add)
        if node_ref == self.head:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        else:
            node_ref.prev.next = new_node
            new_node.prev = node_ref.prev
            new_node.next = node_ref
            node_ref.prev = new_node
    
    def remove_node_by_ref(self, node):
        if node == self.head:
            node.next.prev = None
            self.head = node.next
            del node
        else:
            node.next.prev = node.prev
            node.prev.next = node.next
            del node
    
    def find_value(self, value):
        for node in self:
            if node.data == value: break
        return node

# Classe para Heap

In [8]:
class MinHeap(object):
    
    def __init__(self, modo=None, array=None):
        if modo == None:
            self.dados = []
        if modo == 'array':
            self.Heapfy(array)
    
    def Ajustar_para_baixo(self, i):
        left_child = 2 * i + 1
        right_child = 2 * i + 2
        length = len(self.dados) - 1
        smallest = i
        if left_child <= length and self.dados[i] > self.dados[left_child]:
            smallest = left_child
        if right_child <= length and self.dados[smallest] > self.dados[right_child]:
            smallest = right_child
        if smallest != i:
            self.dados[i], self.dados[smallest] = self.dados[smallest], self.dados[i]
            self.Ajustar_para_baixo(smallest)
    
    def Ajustar_para_cima(self, i):
        parent = math.floor(i / 2)
        length = len(self.dados) - 1
        if parent >= 0 and self.dados[parent] > self.dados[i]:
            self.dados[i], self.dados[parent] = self.dados[parent], self.dados[i]
            self.Ajustar_para_cima(parent)
    
    def Heapfy(self, array):
        self.dados = array.copy()
        for i in reversed(range(len(array)//2)):
            self.Ajustar_para_baixo(i)
    
    def GetMin(self):
        return self.dados[0]
    
    def PopMin(self):
        self.dados[0], self.dados[-1] = self.dados[-1], self.dados[0]
        minimo = self.dados.pop()
        self.Ajustar_para_baixo(0)
        return minimo 
    
    def InsertItem(self, value):
        self.dados.append(value)
        self.Ajustar_para_cima(len(self.dados) - 1)
    
    def PopItem(self, index):
        self.dados[index], self.dados[-1] = self.dados[-1], self.dados[index]
        self.dados.pop()
        if index < len(self.dados):
            self.Ajustar_para_cima(index)
            self.Ajustar_para_baixo(0)
        return
    
    def HeapSort(self, array):
        self.dados = array.copy()
        self.Heapfy(self.dados)
        sorted_array = []
        for _ in range(len(self.dados)):
            self.dados[0], self.dados[-1] = self.dados[-1], self.dados[0]
            sorted_array.append(self.dados.pop())
            self.Ajustar_para_baixo(0)
        return sorted_array

# Classe para árvore de busca binária

In [9]:
class NodeArvoreBinaria(object):
    """
    documentação da classe
    """
    def __init__(self, label, key, h_left=None, h_right=None):
        
        self.label = label
        self.key = key
        self.index = None
        self.parent = None
        self.left_child = None
        self.right_child = None
        if h_left != None: self.h_left = h_left
        if h_right != None: self.h_right = h_right
    
    def CopiarNode(self):
        key = self.GetKey()
        label = self.GetLabel()
        right_child = self.GetRightChild()
        left_child = self.GetLeftChild()
        parent = self.GetParent()
        
        novo_node = NodeArvoreBinaria(label, key)
        
        novo_node.right_child = right_child
        novo_node.left_child = left_child
        novo_node.parent = parent
        
        return novo_node
    
    def GetKey(self):
        if self == None: return None
        return self.key
    
    def SetKey(self, key):
        self.key = key
    
    def GetLabel(self):
        if self == None: return None
        return self.label
    
    def SetLabel(self, label):
        self.label = label
    
    def GetLeftChild(self):
        return self.left_child
    
    def SetLeftChild(self, child):
        self.left_child = child
    
    def GetRightChild(self):
        return self.right_child
    
    def SetRightChild(self, child):
        self.right_child = child
    
    def GetParent(self):
        return self.parent
    
    def SetParent(self, parent):
        self.parent = parent
    
    def Copiar_Dados(self, target):
        self.SetKey(target.GetKey())
        self.SetLabel(target.GetLabel())
    
    def Copiar_Filhos(self, target):
        self.SetLeftChild(target.GetLeftChild())
        self.SetRightChild(target.GetRightChild())
    
    def Deletar_Right_Child(self):
        filho = self.GetRightChild()
        self.right_child.SetParent(None)
        self.SetRightChild(None)
        del filho
    
    def Deletar_Left_Child(self):
        filho = self.GetLeftChild()
        self.left_child.SetParent(None)
        self.SetLeftChild(None)
        del filho
    

class Arvore_Binaria_de_Busca(object):
    """
    documentação da classe
    """
    def __init__(self):
        self.tamanho = 0
        self.root = None
    
    def GetRoot(self):
        return self.root
    
    def SetRoot(self, root):
        self.root = root
    
    def DeletarLeaf(self, node):
        if node.GetParent() == None:
            self.SetRoot(None)
            self.tamanho = 0
            del node
            return
        else:
            self.tamanho -= 1
            if node.GetKey() <= node.parent.GetKey():
                node.parent.Deletar_Left_Child()
            elif node.GetKey() > node.parent.GetKey():
                node.parent.Deletar_Right_Child()
            del node
    
    def DeletarPaiDeFilhoUnico(self, node):
        if node.GetKey() == self.root.GetKey():
            if node.GetLeftChild() != None:
                node.left_child.SetParent(None)
                self.SetRoot(node.left_child)
            elif node.GetRightChild() != None:
                node.right_child.SetParent(None)
                self.SetRoot(node.right_child)
        else:
            if node.GetKey() <= node.parent.GetKey():
                if node.GetLeftChild() != None:
                    node.left_child.SetParent(node.parent)
                    node.parent.SetLeftChild(node.left_child)
                elif node.GetRightChild() != None:
                    node.right_child.SetParent(node.parent)
                    node.parent.SetLeftChild(node.right_child)
            if node.GetKey() > node.parent.GetKey():
                if node.GetLeftChild() != None:
                    node.left_child.SetParent(node.parent)
                    node.parent.SetRightChild(node.left_child)
                elif node.GetRightChild() != None:
                    node.right_child.SetParent(node.parent)
                    node.parent.SetRightChild(node.right_child)
        del node
        self.tamanho -= 1
    
    def DeletarPaiDeDois(self, node, sucessor):
        if sucessor == node.right_child:
            sucessor.SetParent(node.parent)
            sucessor.SetLeftChild(node.left_child)
            node.left_child.SetParent(sucessor)
            
            if node.GetKey() == self.root.GetKey():
                self.SetRoot(sucessor)
            else:
                if node.GetKey() <= node.parent.GetKey():
                    node.parent.SetLeftChild(sucessor)
                elif node.GetKey() > node.parent.GetKey():
                    node.parent.SetRightChild(sucessor)
        else:
            if sucessor.GetRightChild() != None:
                sucessor.right_child.SetParent(sucessor.parent)
                sucessor.parent.SetLeftChild(sucessor.right_child)
            else:
                sucessor.parent.SetLeftChild(None)
            
            sucessor.SetParent(node.parent)
            sucessor.SetLeftChild(node.left_child)
            node.left_child.SetParent(sucessor)
            sucessor.SetRightChild(node.right_child)
            node.right_child.SetParent(sucessor)
            
            if node.GetKey() == self.root.GetKey():
                self.SetRoot(sucessor)
            else:
                if node.GetKey() <= node.parent.GetKey():
                    node.parent.SetLeftChild(sucessor)
                elif node.GetKey() > node.parent.GetKey():
                    node.parent.SetRightChild(sucessor)
        self.tamanho -= 1
        del node
    
    def Busca(self, valor, start='root'):
        
        # se nao for especificado o início da busca, começar pela raíz
        if start == 'root': start = self.GetRoot()
        
        # se não tiver raíz, retorna None
        if start == None:
            return 'arvore vazia'
        
        # se valor encotrado, retornar
        if start.GetKey() == valor:
            res_busca = {'found': True, 'node':start, 'key': start.GetKey(), 'label': start.GetLabel(), 'indice': start.index, 'parent': start.GetParent()}
            return res_busca
        
        # se valor não encontrado
        else:
            if valor < start.GetKey():
                start = start.GetLeftChild()
                if start == None:
                    res_busca = {'found': False, 'parent': start}
                    found = True
                    return res_busca
                else:
                    return self.Busca(valor, start=start)
            else:
                if start.GetRightChild() == None:
                    res_busca = {'found': False, 'parent': start}
                    found = True
                    return res_busca
                else:
                    start = start.GetRightChild()
                    return self.Busca(valor, start=start)
    
    def Inserir(self, label, key, start='root'):
        
        if start == 'root': start = self.root
        
        novo_node = NodeArvoreBinaria(label, key)
        
        # se a arvore está vazia
        if self.root == None:
            self.SetRoot(novo_node)
            novo_node.index = 0
            self.tamanho = 1
        # se a árvore já tem, pelo menos, a raíz
        else:
            if key <= start.GetKey():
                if start.GetLeftChild() == None:
                    start.SetLeftChild(novo_node)
                    novo_node.SetParent(start)
                    novo_node.index = start.index + 1
                    self.tamanho += 1
                else:
                    self.Inserir(label, key, start=start.left_child)
            elif key > start.GetKey():
                if start.GetRightChild() == None:
                    start.SetRightChild(novo_node)
                    novo_node.SetParent(start)
                    novo_node.index = start.index + 1
                    self.tamanho += 1
                else:
                    self.Inserir(label, key, start=start.right_child)
    
    def GetMin(self):
        if self.GetRoot() == None: return None
        start = self.root
        minimo = self.root.GetKey()
        min_node = start
        while start.GetLeftChild() != None:
            start = start.left_child
            if start.GetKey() < minimo:
                minimo = start.GetKey()
                min_node = start
        return min_node, minimo
    
    def GetMax(self):
        if self.GetRoot() == None: return None
        start = self.root
        maximo = self.root.GetKey()
        max_node = start
        while start.GetRightChild() != None:
            start = start.right_child
            if start.GetKey() > maximo:
                maximo = start.GetKey()
                max_node = start
        return max_node, maximo
    
    def GetSucessor(self, start):
        if start.GetRightChild() == None: return None
        sucessor = start.right_child
        while sucessor.GetLeftChild() != None:
            sucessor = sucessor.left_child
        return (sucessor, sucessor.GetKey())
    
    def Deletar(self, valor, start='root'):
        
        if start == 'root': start = self.root
        
        # se não achar valor
        if start == None: return start
        
        # se tem que ir pra esquerda
        if valor < start.GetKey():
            self.Deletar(valor, start=start.left_child)
        # se tem que ir pra direita
        elif valor > start.key:
            self.Deletar(valor, start.right_child)
        # se encontrou o valor, deletar
        else:
            # caso 1: se node já está numa folha, simplesmente remover
            if start.GetLeftChild() == None and start.GetRightChild() == None:
                self.DeletarLeaf(start)
                return
            
            # caso 2: se só tem um filho, filho substitui
            elif not(start.GetLeftChild() != None and start.GetRightChild() != None):
                self.DeletarPaiDeFilhoUnico(start)
                return
            
            # caso 3: se tem dois filhos, achar sucessor e substituir pelo node a ser deletado
            else:
                sucessor = self.GetSucessor(start)[0]
                self.DeletarPaiDeDois(start, sucessor)
                return
    
    def PopMin(self):
        min_node, minimo = self.GetMin()
        label = min_node.GetLabel()
        self.Deletar(minimo)
        return {'label': label, 'key': minimo}
    
    def PopMax(self):
        max_node, maximo = self.GetMax()
        label = max_node.GetLabel()
        self.Deletar(maximo)
        return {'label': label, 'key': maximo}

# Outras estruturas de dados

In [10]:
import Fibonacci_Heap

In [11]:
import Arvore_AVL

## Implementação da heurística de construção

In [12]:
def ConstruirSolucao(dados, n_projetos, n_sondas, n_periodos, custo_total, desloc, criterio='lucro/custo'):
    
    import math
    import copy
    import numpy as np
    import heapq
    
    lag = n_sondas
    
    # inicializando solução
    s = {i:DoublyLinkedList(nodes=[[-1, 0, n_periodos]]) for i in range(n_sondas)}
    
    # inicializando lista de candidatos, por sonda
    s_candidatos = {i:[] for i in range(n_sondas)}
    
    # armazena lista de candidatos como uma heap, dependendo do critério guloso
    for i in range(n_sondas):
        for j in range(n_projetos):
            if (criterio == 'lucro'):
                criterio_val = dados[19][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'lucro/custo'):
                criterio_val = dados[19][j] / dados[20][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'lucro/duracao'):
                criterio_val = dados[19][j] / dados[21][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'custo'):
                criterio_val = dados[20][j]
                heapq.heappush( s_candidatos[i], (criterio_val, j) )
            elif (criterio == 'duracao'):
                criterio_val = dados[21][j]
                heapq.heappush( s_candidatos[i], (criterio_val, j) )
    
    # inicializa contador de gastos total
    gastos = 0
    
    # inicializa conjunto de projetos já selecionados
    proj_usados = set()
    
    # conjunto de sondas é representado com uma Heap
    sondas = [(-n_periodos, i) for i in range(n_sondas)]
    
    # inicializa valor da função objetivo da solução
    fitness = 0
    
    # enquanto nenhuma restrição for alcançada
    while (gastos < custo_total and sondas):
        
        # escolher uma sonda
        (tam_prev, sonda) = heapq.heappop(sondas)
        
        # escolher um projeto
        projeto = heapq.heappop(s_candidatos[sonda])[1]
        
        # se valor do projeto não aumenta a função objetivo, pular
        if (dados[19][projeto] <= 0.):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        # se projeto já foi escolhido, pular
        if (projeto in proj_usados):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        # se executar o projeto excede limite de gastos, pular
        if (gastos + dados[20][projeto] > custo_total):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        aloc = False
        proc = dados[21][projeto]
        
        # percorre atividades da sonda para ver se existe janela disponível na sonda
        for i, node in enumerate(s[sonda]):
            
            # se node não for de janela de tempo, pular
            if (node.data[0] != -1):
                continue
            
            # calcula tempo de deslocamento / setup =====================
            # se node for head, considerar origem da sonda
            if node.prev == None:
                setup = desloc[sonda][projeto + lag]
            # senão, considerar projeto anterior
            else:
                setup = desloc[node.prev.data[0] + lag][projeto + lag]
            
            # calcula o mínimo tempo de início e o máximo tempo de término do projeto dentro da janela
            release = max(node.data[1], dados[22][projeto])
            due = min(node.data[2], dados[23][projeto])
            
            # se consigo colocar o projeto dentro da janela
            if (release + proc + setup <= due):
                
                # escolher tempo de inicio para alocar projeto
                delta1 = (due - release) - (proc + setup) + (node.data[2] - due)
                delta2 = (due - release) - (proc + setup) + (release - node.data[1])
                if (delta1 >= delta2):
                    inicio = copy.copy(release)
                    final = copy.copy(release + setup + proc)
                else:
                    inicio = copy.copy(due - proc - setup)
                    final = copy.copy(due)
                
                # alocar
                proj_usados.add(copy.copy(projeto))
                fitness += dados[19][projeto]
                aloc = True
                
                # atualizando a lista de atividades ========================================
                # se projeto preenche janela toda
                if ( (int(node.data[1])==int(inicio)) and (int(node.data[2])==int(final)) ):
                    
                    # substituir janela pela alocação do projeto
                    node.data = [copy.copy(projeto), inicio, final]
                
                # senão, se projeto preenche início da janela    
                elif (int(node.data[1])==int(inicio)):
                    
                    # atualizar release da janela
                    node.data[1] = final
                    
                    # inserir projeto antes da janela
                    s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)
                
                # senão, se projeto preenche final da janela
                elif (int(node.data[2])==int(final)):
                    
                    # atualiza a due da janela
                    node.data[2] = inicio
                    
                    # inserir projeto depois da janela
                    s[sonda].add_node_after_node(value_to_add=[projeto, inicio, final], node_ref=node)
                
                # senão, se projeto preenche o meio da janela
                else:
                    
                    # atualiza release da janela
                    temp = copy.copy(node.data[1])
                    node.data[1] = final
                    
                    # insere janela nova antes da janela original
                    s[sonda].add_node_before_node(value_to_add=[-1, temp, inicio], node_ref=node)
                    
                    # insere projeto antes da janela original
                    s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)
                    
                break
            else:
                # senão, tentar realocar projetos ====================================================
                
                # verificar se janela da sonda tem interceção com janela do projeto
                if ((node.data[1] < dados[23][projeto]) or (node.data[2] > dados[22][projeto])):
                    
                    # consigo antecipar o prev? quanto?
                    delta_prev = 0
                    if node.prev == None:
                        continue
                    elif node.prev.prev == None:
                        continue
                    else:
                        if node.prev.prev.data[0] != -1:
                            continue
                        else:
                            delta_prev = node.prev.data[1] - max(node.prev.prev.data[1], dados[22][node.prev.data[0]])
                    
                    # consigo postergar o next? quanto?
                    delta_next = 0
                    if node.next == None:
                        continue
                    if node.next.next == None:
                        continue
                    else:
                        if node.next.next.data[0] != -1:
                            continue
                        else:
                            delta_next = min(node.next.next.data[2], dados[23][node.next.data[0]]) - node.next.data[2]
                    
                    # quanto de delta eu preciso?
                    delta_min = release + proc + setup - due
                    
                    # só com antecipação do prev, torna viável?
                    if (delta_prev >= delta_min ):
                        # fazer antecipação do prev
                        node.prev.data[1] -= delta_min
                        node.prev.data[2] -= delta_min
                        node.prev.prev.data[2] -= delta_min
                        aloc = True
                        node.data[1] -= delta_min
                    # só com postergação do next, torna viável?
                    elif (delta_next >= delta_min):
                        # fazer postergação do next
                        node.next.data[1] += delta_min
                        node.next.data[2] += delta_min
                        node.next.next.data[1] += delta_min
                        aloc = True
                        node.data[2] += delta_min
                    # somando antecipação do prev e postergação do next, torna viável?
                    elif (delta_prev + delta_next >= delta_min):
                        # fazer antecipação do prev
                        node.prev.data[1] -= delta_prev
                        node.prev.data[2] -= delta_prev
                        node.prev.prev.data[2] -= delta_prev
                        node.data[1] -= delta_min
                        # fazer postergação do next
                        node.next.data[1] += (delta_min - delta_prev)
                        node.next.data[2] += (delta_min - delta_prev)
                        node.next.next.data[1] += (delta_min - delta_prev)
                        aloc = True
                        node.data[2] += delta_min
                        
                    # alocar projeto
                    if aloc:
                        node.data[0] = projeto
                        proj_usados.add(copy.copy(projeto))
                        fitness += dados[19][projeto]
                        break
        
        # se projeto foi alocado, atualizar dados
        if (aloc):
            gastos += dados[20][projeto]
            # se sonda ainda tem candidatos, colocar de volta na heap com valor atualizado
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, ( (tam_prev + (final-inicio+1)) , sonda))
        # senão, apenas recolocar a sonda
        else:
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
    
    return fitness, gastos, s, s_candidatos

In [13]:
def VerificaRestricaoOrcamento():
    
    """
    Documentação:
    """
    
    return

In [14]:
def VerificaRestricaoUnicidadeProjetos():
    
    """
    Documentação:
    """
    
    return

In [15]:
def VerificaRestricaoTimeWindows():
    
    """
    Documentação:
    """
    
    return

In [16]:
def VerificaRestricaoScheduling():
    
    """
    Documentação:
    """
    
    return

In [17]:
def VerificaRestricoes():
    
    """
    Documentação:
    """
    
    return

In [18]:
def RodarHeuristica(arquivo, criterio='lucro/custo', print_sol=True):
    
    """
    Documentação:
    """
    
    import CPUtimer
    import copy
    
    n_projetos, n_sondas, n_periodos, t_init, t_final, delta_t, custo_total, dados, desloc = Read_data(path=arquivo)
    
    timer = CPUtimer.CPUTimer()
    timer.reset()
    timer.start()
    
    f, gastos , s, _ = ConstruirSolucao(dados, n_projetos, n_sondas, n_periodos, custo_total, desloc, criterio=criterio)
    
    tempo_total = timer.get_time(reference = "total", unit = "seconds")
    tempo_medio = timer.get_time(reference = "average", unit = "seconds")
    
    if print_sol:
        print ('fitness: ', f)
        print ('gastos: ', gastos)
        print ('tempo total de execução (segundos) : ', tempo_total)
        print ('tempo médio de execução (segundos) ', tempo_medio)
        for i in range(n_sondas):
            for node in s[i]:
                print ('sonda ', i, ' executa projetos: ', node.data)
    
    return f, gastos, s, tempo_total, tempo_medio

In [19]:
def RodarHeuristicaSensibilidade(diretorio, criterios=None):
    
    """
    Documentação:
    """
    
    from os import walk
    import copy
    import pandas as pd
    
    if criterios == None:
        criterios = ['lucro', 'lucro/custo', 'lucro/duracao', 'custo', 'duracao']
    
    resultados = {}
    resultados['Instancia'] = []
    resultados['Criterio guloso'] = []
    resultados['Tempo total (s)'] = []
    resultados['Tempo medio (s)'] = []
    resultados['Valor da função objetivo'] = []
    resultados['Viável?'] = []
    
    arquivos = _, _, arquivos = next(walk(diretorio))
    arquivos = ["./instancias/" + f for f in arquivos]
    
    for f in arquivos:
        for c in criterios:
            fitness, gastos, s, t_total, t_medio = RodarHeuristica(arquivo=f, criterio=c, print_sol=False)
            
            resultados['Instancia'].append(copy.deepcopy(f.split(diretorio)[1]))
            resultados['Criterio guloso'].append(copy.deepcopy(c))
            resultados['Tempo total (s)'].append(copy.deepcopy(t_total))
            resultados['Tempo medio (s)'].append(copy.deepcopy(t_medio))
            resultados['Valor da função objetivo'].append(copy.deepcopy(fitness))
            resultados['Viável?'].append(copy.deepcopy(True))
    
    df = pd.DataFrame(resultados)
    
    return resultados, df

In [20]:
ls instancias/

instancia_100projetos_10sondas_delta_t28.dat
instancia_100projetos_10sondas_delta_t7.dat
instancia_100projetos_10sondas_delta_t84.dat
instancia_100projetos_2sondas_delta_t1.dat
instancia_100projetos_2sondas_delta_t28.dat
instancia_100projetos_2sondas_delta_t7.dat
instancia_100projetos_2sondas_delta_t84.dat
instancia_100projetos_5sondas_delta_t28.dat
instancia_100projetos_5sondas_delta_t7.dat
instancia_100projetos_5sondas_delta_t84.dat
instancia_10projetos_2sondas_delta_t1.dat
instancia_10projetos_2sondas_delta_t28.dat
instancia_10projetos_2sondas_delta_t7.dat
instancia_10projetos_2sondas_delta_t84.dat
instancia_200projetos_10sondas_delta_t28.dat
instancia_200projetos_10sondas_delta_t7.dat
instancia_200projetos_10sondas_delta_t84.dat
instancia_200projetos_20sondas_delta_t28.dat
instancia_200projetos_20sondas_delta_t7.dat
instancia_200projetos_20sondas_delta_t84.dat
instancia_200projetos_2sondas_delta_t1.dat
instancia_200projetos_2sondas_delta_t28.dat
instancia_200projetos_2sondas_delta_

In [21]:
arq = 'instancias/instancia_300projetos_20sondas_delta_t7.dat'

In [ ]:
%%time

c = 'lucro'

f, gastos, s, tempo, t_medio = RodarHeuristica(arquivo=arq, criterio=c, print_sol=True)

In [ ]:
%%time

c = 'lucro/custo'

f, gastos, s, tempo, t_medio = RodarHeuristica(arquivo=arq, criterio=c, print_sol=True)

In [ ]:
%%time

c = 'lucro/duracao'

f, gastos, s, tempo, t_medio = RodarHeuristica(arquivo=arq, criterio=c, print_sol=True)

In [ ]:
%%time

c = 'custo'

f, gastos, s, tempo, t_medio = RodarHeuristica(arquivo=arq, criterio=c, print_sol=True)

In [ ]:
%%time

c = 'duracao'

f, gastos, s, tempo, t_medio = RodarHeuristica(arquivo=arq, criterio=c, print_sol=True)

In [ ]:
%%time

d = "./instancias"

resultados, df_resultados = RodarHeuristicaSensibilidade(diretorio=d, criterios=None)

In [ ]:
df_resultados

In [ ]:
df_resultados.to_csv(path_or_buf='resultados.csv', header=True, index=False, decimal='.')